# Training

## Loading data
The first step is to load the training dataset.

In [1]:
from time import time, localtime, strftime
t00 = time()
import numpy as np

In [2]:
X_train = './data/X_reference.npy'
y_train = './data/y_reference.npy'
X = np.load(X_train)
y = np.load(y_train)
print(X.shape, y.shape)

(60000, 1000) (60000,)


## Loading CNN model


In [3]:
from resnet import ResNet
import os
import torch

In [4]:
# CNN parameters
layers = 6
hidden_size = 100
block_size = 2
hidden_sizes = [hidden_size] * layers
num_blocks = [block_size] * layers
input_dim = 1000
in_channels = 64
n_classes = 30
os.environ['CUDA_VISIBLE_DEVICES'] = '{}'.format(0)
cuda = torch.cuda.is_available()

In [5]:
cnn = ResNet(hidden_sizes, num_blocks, input_dim=input_dim,
                in_channels=in_channels, n_classes=n_classes)
if cuda: cnn.cuda()


## Training


In [6]:
from datasets import spectral_dataloader
from training import run_epoch
from torch import optim

### Train/val split
We split the training dataset into train and validation sets. We randomly sample 10% of the dataset to use as a validation set.

In [7]:
p_val = 0.1
n_val = int(60000 * p_val)
idx_tr = list(range(60000))
np.random.shuffle(idx_tr)
idx_val = idx_tr[:n_val]
idx_tr = idx_tr[n_val:]

# Training CNN
epochs = 30 # Change this number to ~30 for full training
batch_size = 500
t0 = time()
# Set up Adam optimizer
optimizer = optim.Adam(cnn.parameters(), lr=1e-3, betas=(0.5, 0.999))
# Set up dataloaders
dl_tr = spectral_dataloader(X, y, idxs=idx_tr,
    batch_size=batch_size, shuffle=True)
dl_val = spectral_dataloader(X, y, idxs=idx_val,
    batch_size=batch_size, shuffle=False)

In [8]:

# Fine-tune CNN for first fold
best_val = 0
no_improvement = 0
max_no_improvement = 5
print('Starting training!')
for epoch in range(epochs):
    print(' Epoch {}: {:0.2f}s'.format(epoch+1, time()-t0))
    # Train
    acc_tr, loss_tr = run_epoch(epoch, cnn, dl_tr, cuda,
        training=True, optimizer=optimizer)
    print('  Train acc: {:0.2f}'.format(acc_tr))
    # Val
    acc_val, loss_val = run_epoch(epoch, cnn, dl_val, cuda,
        training=False, optimizer=optimizer)
    print('  Val acc  : {:0.2f}'.format(acc_val))
    # Check performance for early stopping
    if acc_val > best_val or epoch == 0:
        best_val = acc_val
        no_improvement = 0
        best_model = cnn
    else:
        no_improvement += 1
    if no_improvement >= max_no_improvement:
        print('Finished after {} epochs!'.format(epoch+1))
        break

torch.save(best_model.state_dict()
           ,'pretrained_'+strftime('%y-%m-%d-%H-%M',localtime(t00))+'.ckpt')
NA
print('\n completed in: {:0.2f}s'.format(time()-t00))

Starting training!
 Epoch 1: 0.00s


KeyboardInterrupt: 

In [8]:
from training import get_predictions
import numpy as np

<All keys matched successfully>

In [9]:
X_test = './data/X_test.npy'
y_test = './data/y_test.npy'
X = np.load(X_test)
y = np.load(y_test)
print(X.shape, y.shape)
dl_test = spectral_dataloader(X, y, batch_size=batch_size, shuffle=True)

(3000, 1000) (3000,)


In [10]:
preds_tr, corr_tr = get_predictions(best_model,dl_tr,cuda)
preds_val, corr_val = get_predictions(best_model,dl_val,cuda)
preds_test, corr_test = get_predictions(best_model,dl_test,cuda)
print("train accuracy = {}%".format(corr_tr/54000*100))
print("validation accuracy = {}%".format(corr_val/6000*100))
print("test accuracy = {}%".format(corr_test/3000*100))

496
982
1475
1969
2464
2954
3449
3944
4434
4930
5424
5921
6412
6908
7403
7895
8388
8881
9374
9861
10355
10850
11343
11837
12328
12819
13316
13811
14298
14791
15283
15776
16267
16763
17254
17747
18240
18735
19225
19721
20213
20705
21197
21689
22182
22677
23176
23669
24160
24655
25148
25641
26132
26625
27117
27611
28105
28597
29090
29581
30076
30568
31060
31554
32047
32543
33032
33526
34018
34510
34999
35490
35983
36473
36964
37455
37943
38436
38926
39420
39912
40407
40896
41393
41887
42382
42876
43364
43854
44346
44841
45329
45822
46314
46810
47304
47797
48289
48785
49280
49772
50261
50756
51250
51740
52235
52728
53224
496
985
1476
1968
2461
2953
3448
3946
4438
4928
5425
5923
282
567
863
1128
1407
1690
train accuracy = 98.56296296296296%
validation accuracy = 98.71666666666667%
test accuracy = 56.333333333333336%
